In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import imp # for reloading modules

import helper as h

%matplotlib inline

In [136]:
imp.reload(h)

<module 'helper' from '/Users/dirkkalmbach/Library/Mobile Documents/com~apple~CloudDocs/Projekte/DSND/blog-project/helper.py'>

# Problem Understanding
## Definition of Target Group(s)
The aim of this Report is to find out how Data Scientists can work independently, i.e. as Freelancer, independent contractor, or 1-person-company.
I will use the following abbreviations:
> - `Pure Data Scientists` (PDS): Data Scientists, who consider himself/herself as **only** _Data Scientists / Machine Learning Specialists_
> - `Data Scientist` (DS): Respndents who consider him/herself as *Data Scientist*, but **not only**. 
> - `Data Jobs` (DJ): 
_Data Scientists_ 👆🏻, and/or _Data/Business Anaylsts_ and/or _Data Engineers_, i.e.: Respondents who did not mark any other "

## Data Science Question 
1. Are independent Data Scientists more happy with their Job than employed Data Scientists?
2. Do indpeendent Data Scientists earn more than employed Data Scientists?
3. What Coding Languages use Data Scientists? Is here any development over time?
...
4. What makes Developer in general happier? (also DevType as Dummy)

# Data Understanding

## Load Data

In [147]:
#TODO: refactor: unzip_dataframe() in helper

# Load survey 2019 dataset (unzip first)
file = "data/raw data/developer_survey_2019/survey_results_public.csv"

from zipfile import ZipFile

zip_file = ZipFile('data/raw data/developer_survey_2019/survey_results_public.csv.zip')
df = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df = df["survey_results_public.csv"]

# Load data description
schema = pd.read_csv("data/raw data/developer_survey_2019/survey_results_schema.csv")

h.df_info(df)

Rows: 88883 	 Variables: 85 	 	 Size: n/a kb


## Definition of Target Group(s)

In [148]:
h.get_desc("Employment")

'Which of the following best describes your current employment status?'

In [149]:
# Im only interested in full working people -> no parttime, students, etc.
df = df[(df["Employment"]=='Employed full-time') | (df["Employment"]=='Independent contractor, freelancer, or self-employed')]
h.df_info(df)

Rows: 72951 	 Variables: 85 	 	 Size: n/a kb


In [124]:
# Freelancing is asked 2x -> I wonder if there are some mismatches
df[ (df["Employment"]=='Independent contractor, freelancer, or self-employed') & (
     df["OrgSize"]== "Just me - I am a freelancer, sole proprietor, etc.")
  ].shape

(3817, 87)

#### How many "Pure" Data Scientists are in the dataset?

In [69]:
PDS = df[df["DevType"]=="Data scientist or machine learning specialist"]
h.df_info(PDS)

Rows: 494 	 Variables: 86 	 	 Size: Nonekb


_...and how many among them are Freelancer?_

In [70]:
# How many Freelance Pure Data Scientists?
h.df_info(
    PDS[PDS["Employment"]=="Independent contractor, freelancer, or self-employed"]
)

Rows: 25 	 Variables: 86 	 	 Size: Nonekb


Wow! 😱 Only 25 Cases! Of course, this is not enough for any valuable insights 😫

In [174]:
# What about Freelance Data Scientists?

DJ=["Data scientist or machine learning specialist", "Data or business analyst", "Engineer, data"]
RS=["Academic Research","Scientist"]
DS=["Data scientist or machine learning specialist"]

def t(col, jobs, cat):
    found=False
    for job in jobs:
        if job in col:
            found=True
    if found:
        return cat
    else:
        return col
            
            
# Science Jobs
df["DataJob"]=df["DevType"].apply(lambda x: t(str(x),RS,"Something with Science"))

# Data Jobs
df["DataJob"]=df["DevType"].apply(lambda x: t(str(x),DJ,"Something with Data"))

# Data Scientists
df["DataJob"]=df["DevType"].apply(lambda x: t(str(x),DS,"Data Scientist"))


df.groupby("DataJob").count()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase,JobSatIndex
DataJob,,,,,,,,,,,,,,,,,,,,,
Academic researcher,240,234,240,240,228,240,240,232,236,210,...,162,173,215,204,178,191,208,225,227,190
Academic researcher;Data or business analyst,14,14,14,14,14,14,14,12,14,10,...,11,10,14,12,7,10,13,13,13,12
Academic researcher;Data or business analyst;Database administrator,3,3,3,3,3,3,3,3,2,2,...,3,1,3,3,2,2,1,2,2,2
"Academic researcher;Data or business analyst;Database administrator;Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, front-end;Developer, QA or test;Educator;Scientist",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Academic researcher;Data or business analyst;Database administrator;Designer;Developer, back-end;Developer, desktop or enterprise applications;Developer, embedded applications or devices;Developer, front-end;Developer, full-stack;DevOps specialist",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Senior executive/VP;System administrator,2,2,2,2,2,2,2,2,2,2,...,2,0,2,2,2,2,2,2,2,1
Student,335,330,335,335,324,335,335,319,317,184,...,286,275,317,310,263,280,304,326,326,34
Student;System administrator,18,18,18,18,17,18,18,18,18,8,...,16,17,18,18,17,17,17,18,18,7


In [179]:
df[df["DataJob"]=="Something with Data"].shape

(0, 87)

## Job Satisfaction

How happy are Data Scientists with their job?

In [14]:
# Question in Survey 2019
h.get_desc("JobSat")

'How satisfied are you with your current job? (If you work multiple jobs, answer for the one you spend the most hours on.)'

In [150]:
# build jobsatisfaction index (1-5)

index = {'Slightly satisfied': 4, 
         'Slightly dissatisfied': 2,
         'Neither satisfied nor dissatisfied': 3,
         'Very satisfied': 5,
         'Very dissatisfied': 1,
         np.nan: np.nan
        }

df["JobSatIndex"] = df["JobSat"].apply(lambda x: index[x])

print("Avg Job Satisfaction: ", df.JobSatIndex.mean().round(2))  

Avg Job Satisfaction:  3.69


In [156]:
df.groupby('Employment') \
       .agg(n=('Employment', 'size'), Job_Satisfaction=('JobSatIndex', 'mean')).reset_index()

,Employment,n,Job_Satisfaction
0,Employed full-time,64440,3.684898
1,"Independent contractor, freelancer, or self-em...",8511,3.713764


> ❗__Interesting__: no difference between Freelancer and employed